In [4]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [20]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'


In [21]:
!pip install boto3
import boto3
import os
import json
!pip install mistralai
from mistralai import Mistral
from google.colab import userdata

mistral_api_key = userdata.get('MISTRAL_API_KEY')
s3 = boto3.client('s3')
bucket_name = 'hydroponic123'
prefix = 's3://hydroponic123/bad-strawberry1.jpg'
client = Mistral(api_key=mistral_api_key)

In [22]:
def list_images_in_s3(bucket, prefix=''):
    """List image files in an S3 bucket with a prefix"""
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    return [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].lower().endswith(('.jpg', '.png'))]

def get_presigned_url(bucket, key, expiration=3600):
    """Generate a temporary URL to access the S3 object"""
    return s3.generate_presigned_url('get_object',
                                     Params={'Bucket': bucket, 'Key': key},
                                     ExpiresIn=expiration)


In [23]:
def analyze_image(image_url):
    """Send image to Mistral API for analysis"""
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": 'Analyze this strawberry plant image in detail and provide:\n'
                            '1. Overall plant health status\n'
                            '2. Any signs of fungal diseases\n'
                            '3. Fruit condition (if visible)\n'
                            '4. Leaf health\n'
                            '5. Specific disease identification (if any)\n'
                            '6. Recommended actions\n'
                            'Format the response as a JSON object with these keys:\n'
                            'health_status, fungal_status, fruit_condition, leaf_health,\n'
                            'diseases_detected, recommendations'
                },
                {
                    "type": "image_url",
                    "image_url": image_url
                }
            ]
        }
    ]

    try:
        chat_response = client.chat.complete(
            model="pixtral-12b-2409",
            messages=messages
        )
        return chat_response.choices[0].message.content
    except Exception as e:
        return {"error": str(e)}


In [24]:

if __name__ == "__main__":
    image_keys = list_images_in_s3(bucket_name, prefix)

    results = []

    for key in image_keys:
        file_name = key.split("/")[-1]
        image_url = get_presigned_url(bucket_name, key)

        print(f"Analyzing image: {file_name}")
        analysis_result = analyze_image(image_url)

        results.append({
            "file_name": file_name,
            "s3_key": key,
            "analysis_result": analysis_result
        })
        print(f"Analysis complete for {file_name}")
        print("-" * 50)

    with open("image_analysis_results.json", "w") as f:
        json.dump(results, f, indent=4)

    print("Results written to image_analysis_results.json")


ClientError: An error occurred (SignatureDoesNotMatch) when calling the ListObjectsV2 operation: The request signature we calculated does not match the signature you provided. Check your key and signing method.